In [88]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# import re
# import pyod
# from pyod.models.iforest import IForest
# import pyod.models.auto_encoder_torch as ae

import sklearn
from sklearn import metrics
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import datetime

In [89]:
#create dictionary of data for each dataset containing 

def process_data(dataset,data_type = 'train'):
    if data_type == 'train':
        grouped_kpi_id = dataset.groupby('KPI ID')
        key_list = list()
        for key,group in grouped_kpi_id:
            key_list.append(key)

        data_dict = {}
        for key in key_list:
            data_dict[key] = grouped_kpi_id.get_group(key).iloc[:,:-1]

        for key in data_dict:
            data_dict[key].reset_index(inplace = True,drop = True)
            data_dict[key]["timestamp"] = pd.to_datetime(data_dict[key]["timestamp"],unit  = 's')
        
        return data_dict,key_list
    else:
        grouped_kpi_id = dataset.groupby('KPI ID')
        key_list = list()
        for key,group in grouped_kpi_id:
            key_list.append(key)

        data_dict = {}
        for key in key_list:
            data_dict[str(key)] = grouped_kpi_id.get_group(key).iloc[:,:-1]

        for key in data_dict:
            data_dict[str(key)].reset_index(inplace = True,drop = True)
            data_dict[str(key)]["timestamp"] = pd.to_datetime(data_dict[str(key)]["timestamp"],unit  = 's')
        key_list_test = [str(key) for key in key_list]
        return data_dict,key_list_test


In [90]:
file_path = os.path.join(os.getcwd(), 'KPI')
file_name = 'train-data.csv'
file_to_load = os.path.join(file_path, file_name)
train_data = pd.read_csv(os.path.join(file_path, file_to_load))

data_dict,key_list_train = process_data(train_data)
model_dict = {}
for key in data_dict:
    data = data_dict[key]
    x_train = np.array(data["value"]).reshape(-1,1)
    y_train = np.array(data["label"]).reshape(-1,1)
    model_dict[key] =  MLPClassifier(hidden_layer_sizes=[100, 100], random_state=1, max_iter=100)
    model_dict[key].fit(x_train,y_train)
    

C:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vect

In [101]:
file_path = os.path.join(os.getcwd(), 'KPI')
file_name = 'test-data.hdf'
file_to_load = os.path.join(file_path, file_name)
test_data = pd.read_hdf(os.path.join(file_path, file_to_load))

data_dict_test,key_list_test = process_data(test_data,data_type='test')
results = np.zeros(shape=(1,2))
for key in key_list_test:
    y_test = np.array(data_dict_test[key]["label"]).reshape(-1,1)
    x_test = np.array(data_dict_test[key]["value"]).reshape(-1,1)
    y_pred_test = model_dict[key_list_test[0]].predict(x_test)
    data = np.hstack((y_test.reshape(-1,1),y_pred_test.reshape(-1,1)))
    results = np.vstack((results,data))



print(classification_report(results[:,0],results[:,1]))

              precision    recall  f1-score   support

         0.0       0.99      0.41      0.58   2864288
         1.0       0.03      0.84      0.05     54560

    accuracy                           0.42   2918848
   macro avg       0.51      0.63      0.32   2918848
weighted avg       0.97      0.42      0.57   2918848



In [102]:
confusion_matrix(results[:,0],results[:,1])

array([[1187207, 1677081],
       [   8863,   45697]], dtype=int64)